# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 1-1
## Учимся переводить между decision/evaluation/search на примере Bin Packing

**Даны:** 
* $[w_1, \dots , w_k], w_i \in [0, 1]$ — веса грузов  
* $n_{\text{bins}}$ — количество корзин грузоподъемностью 1.

Предполагается, что функция `solve_bp_decision(weights, n_bins)` решает «распознавательный» (decision) вариант задачи bin packing. На вход ей подаётся список весов и число корзин. Функция возвращает булевский ответ на вопрос «можно ли заданные веса раскидать по не более чем `n_bins` контейнерам? 

Напишите содержимое функции `solve_bp_evaluation`, которая возвращает оптимальное число корзин (решает evaluation variant задачи bin packing). 

Затем напишите содержимое функции `solve_bp_search`, которая возвращает список номеров корзин, которые при каком-нибудь оптимальном распределении присваиваются весам из списка `weights` (корзины нумеруются с единицы). 

Каждая из следующих функций должна вызывать предыдущую не более чем полиномиальное число раз.

In [9]:
def solve_bp_decision(weights: list, n_bins: int) -> bool:
    def able_to_pack(weights: list, bin_capacities) -> bool:
        return weights == [] or any( 
            able_to_pack( weights[:-1], bin_capacities[:i] + [capacity - weights[-1]] + bin_capacities[(i + 1):] ) 
            for i, capacity in enumerate(bin_capacities) if capacity >= weights[-1] 
        )

    return able_to_pack( sorted(weights), [1.0] * n_bins )

# To test this, run:
solve_bp_decision([0.8, 0.09, 0.4, 0.7], 2)
solve_bp_decision([0.8, 0.09, 0.4, 0.7], 3)

True

In [45]:
def solve_bp_evaluation(weights: list) -> int:
    l = -1
    r = len(weights) + 1
    while r - l > 1:
        m = (l + r + 1) // 2
        if solve_bp_decision(weights, m):
            r = m
        else:
            l = m
    return r

solve_bp_evaluation([0.15, 0.9])

2

In [47]:
def solve_bp_search(weights: list):
    n = solve_bp_evaluation(weights)
    l = len(weights)
    bins = [[x] for x in range(l)]
    while len(weights) > n:
        i = 0
        while i < len(weights):
            j = i + 1
            while j < len(weights):
                if weights[i] + weights[j] <= 1.0 and solve_bp_evaluation(weights[:i] + [weights[i] + weights[j]] + weights[j + 1:]) == n:
                    bins = bins[:i] + [bins[i] + bins[j]] + bins[j + 1:]
                    weights = weights[:i] + [weights[i] + weights[j]] + weights[j + 1:]
                j += 1
            i += 1
    ans = [0] * l
    assert(len(bins) == n)
    for i in range(n):
        for x in bins[i]:
            ans[x] = i + 1
    return ans
    
solve_bp_search([0.9, 0.12, 0.45, 0.5])

[1, 2, 2, 3]